In [1]:
import json

In [2]:
import requests

In [3]:
import geopandas as gpd
import pandas as pd

## Open Geojson

In [4]:
latvia_url = f"https://gis.lgia.gov.lv/arcgis/rest/services/KP_OVERLAYS/Robezas/FeatureServer/9/query?where=1=1&outFields=*&f=geojson"
response = requests.get(latvia_url)

In [5]:
latvia_gdf = gpd.GeoDataFrame.from_features(response.json()["features"])
latvia_gdf.crs = 4326

In [6]:
latvia_gdf = latvia_gdf[["adreses_nosaukums", "geometry", "atvk"]]

## Get Open Population data

In [7]:
url = "https://data.stat.gov.lv:443/api/v1/en/OSP_PUB/START/POP/IR/IRE/IRE031"

In [8]:
data = {
    "query": [
        {"code": "ETHNICITY", "selection": {"filter": "item", "values": ["E_LAT"]}},
        {
            "code": "AREA",
            "selection": {
                "filter": "vs:VS_2021_AREA_LV_RPN_PLAN",
                "values": [
                    "LV",
                    "LV00A",
                    "LV006",
                    "LV007",
                    "LV00C",
                    "LV008",
                    "LV00B",
                    "LV003",
                    "LV009",
                    "LV005",
                    "LV0001000",
                    "LV0002000",
                    "LV0003000",
                    "LV0031010",
                    "LV0004000",
                    "LV0005000",
                    "LV0040010",
                    "LV0006000",
                    "LV0054010",
                    "LV0007000",
                    "LV0020000",
                    "LV0021000",
                    "LV0022000",
                    "LV0023000",
                    "LV0024000",
                    "LV0025000",
                    "LV0026000",
                    "LV0027000",
                    "LV0028000",
                    "LV0029000",
                    "LV0030000",
                    "LV0031000",
                    "LV0032000",
                    "LV0033000",
                    "LV0034000",
                    "LV0035000",
                    "LV0036000",
                    "LV0037000",
                    "LV0038000",
                    "LV0039000",
                    "LV0040000",
                    "LV0041000",
                    "LV0042000",
                    "LV0043000",
                    "LV0044000",
                    "LV0045000",
                    "LV0046000",
                    "LV0047000",
                    "LV0048000",
                    "LV0049000",
                    "LV0051000",
                    "LV0052000",
                    "LV0053000",
                    "LV0054000",
                    "LV0055000",
                    "LV0056000",
                ],
            },
        },
        {"code": "ContentsCode", "selection": {"filter": "item", "values": ["IRE031"]}},
        {"code": "TIME", "selection": {"filter": "item", "values": ["2015", "2025"]}},
    ],
    "response": {"format": "JSON-stat"},
}

In [9]:
r = requests.post(url, data=json.dumps(data))

In [10]:
json_data = r.json()
indexes = json_data["dataset"]["dimension"]["AREA"]["category"]["index"]
labels = json_data["dataset"]["dimension"]["AREA"]["category"]["label"]
data = json_data["dataset"]["value"]

In [11]:
data_2015 = data[0::2]
data_2025 = data[1::2]

In [12]:
data_list = [
    {"name": labels[key], "atvk": key, "2015": data_2015[indx], "2025": data_2025[indx]}
    for key, indx in indexes.items()
]

In [14]:
population_df = pd.DataFrame(data_list)
population_df["atvk"] = population_df["atvk"].str[2:]

## Merge data and export

In [18]:
latvia_population_gdf = latvia_gdf.merge(population_df, on="atvk", how="left")

In [19]:
latvia_population_gdf["change"] = (
    latvia_population_gdf["2025"] - latvia_population_gdf["2015"]
)
latvia_population_gdf["rate"] = (
    (latvia_population_gdf["change"] / latvia_population_gdf["2015"]) * 100
).round(decimals=2)

In [22]:
latvia_population_gdf["change"] = latvia_population_gdf["change"].astype(int)
latvia_population_gdf["rate"] = latvia_population_gdf["rate"].astype(float)

In [23]:
latvia_population_gdf = latvia_population_gdf.to_crs(3059)
latvia_population_gdf.to_file("data/latvian_latvian_population.gpkg")

In [25]:
latvia_population_gdf.sort_values("rate")

,adreses_nosaukums,geometry,atvk,name,2015,2025,change,rate
38,Krāslavas nov.,"POLYGON ((694664.635 222350.528, 694774.36 222...",0032000,Krāslava municipality,13114,10048.0,-3066,-23.38
19,Varakļānu nov.,"POLYGON ((668838.193 275940.374, 668840.077 27...",0055000,Varakļāni municipality,3241,2580.0,-661,-20.39
28,Balvu nov.,"POLYGON ((681453.571 305497.709, 681457.064 30...",0024000,Balvi municipality,17981,14472.0,-3509,-19.52
4,Augšdaugavas nov.,"POLYGON ((684995.68 205168.54, 685021.098 2052...",0022000,Augšdaugava municipality,13089,10534.0,-2555,-19.52
29,Rēzeknes nov.,"POLYGON ((706963.768 234461.549, 707004.092 23...",0043000,Rēzekne municipality,19836,16226.0,-3610,-18.20
25,Ludzas nov.,"POLYGON ((753914.091 268970.042, 753912.704 26...",0037000,Ludza municipality,14930,12252.0,-2678,-17.94
41,Preiļu nov.,"POLYGON ((671526.903 267073.962, 671597.421 26...",0042000,Preiļi municipality,13615,11324.0,-2291,-16.83
26,Alūksnes nov.,"POLYGON ((706130.913 350837.586, 706316.041 35...",0021000,Alūksne municipality,13328,11095.0,-2233,-16.75
30,Madonas nov.,"POLYGON ((592577.129 317364.831, 592593.233 31...",0038000,Madona municipality,28962,24339.0,-4623,-15.96
2,Gulbenes nov.,"POLYGON ((634973.92 350828.68, 634572.039 3508...",0029000,Gulbene municipality,19288,16242.0,-3046,-15.79
